In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torchvision.datasets as datas
import torchvision.transforms as tf
import torchvision.utils as tutils
from torch.autograd import Variable
from PIL import Image
import numpy as np
import torchvision.models as models
import torchvision.models.vgg as VGG

In [0]:
class VGG(nn.Module):
    def __init__(self, pooling):
        super(VGG,self).__init__()
        # VGG structures
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.p1 = pooling(kernel_size=2, stride=2)

        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.p2 = pooling(kernel_size=2, stride=2)

        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv3_4 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.p3 = pooling(kernel_size=2, stride=2)

        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv4_4 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.p4 = pooling(kernel_size=2, stride=2)

        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_4 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.p5 = pooling(kernel_size=2, stride=2)

    def forward(self, x, out_params = None):
        out = {}
        # Building up the VGG net that's going to be used
        out['re11'] = F.relu(self.conv1_1(x))
        out['re12'] = F.relu(self.conv1_2(out['re11']))
        out['p1'] = self.p1(out['re12'])
        h_relu1_2 = out['re12']
        out['re21'] = F.relu(self.conv2_1(out['p1']))
        out['re22'] = F.relu(self.conv2_2(out['re21']))
        out['p2'] = self.p2(out['re22'])
        h_relu2_2 = out['re22']
        out['re31'] = F.relu(self.conv3_1(out['p2']))
        out['re32'] = F.relu(self.conv3_2(out['re31']))
        out['re33'] = F.relu(self.conv3_3(out['re32']))
        out['re34'] = F.relu(self.conv3_4(out['re33']))
        out['p3'] = self.p3(out['re34'])
        h_relu3_3 = out['re33']
        out['re41'] = F.relu(self.conv4_1(out['p3']))
        out['re42'] = F.relu(self.conv4_2(out['re41']))
        out['re43'] = F.relu(self.conv4_3(out['re42']))
        out['re44'] = F.relu(self.conv4_4(out['re43']))
        h_relu4_3 = out['re43']
        out['p4'] = self.p4(out['re44'])
        out['re51'] = F.relu(self.conv5_1(out['p4']))
        out['re52'] = F.relu(self.conv5_2(out['re51']))
        out['re53'] = F.relu(self.conv5_3(out['re52']))
        out['re54'] = F.relu(self.conv5_4(out['re53']))
        out['p5'] = self.p5(out['re54'])
        if out_params is not None:
             return [out[param] for param in out_params]
        vgg_outputs = namedtuple("VggOutputs", ['relu1_2', 'relu2_2', 'relu3_3', 'relu4_3'])
        out = vgg_outputs(h_relu1_2, h_relu2_2, h_relu3_3, h_relu4_3)
        return out



In [0]:
class GramMatrix(nn.Module):
    def forward(self, input):
        b, c, h, w = input.size()
        f = input.view(b, c, h*w) #bxcx(hxw)
        # torch.bmm(batch1, batch2, out=None)
        # batch1 : bxmxp, batch2 : bxpxn -> bxmxn
        G = torch.bmm(f, f.transpose(1, 2)) # f: bxcx(hxw), f.transpose: bx(hxw)xc -> bxcxc
        return G.div_(h*w)

class styleLoss(nn.Module):
    def forward(self, input, target):
        GramInput = GramMatrix()(input)
        return nn.MSELoss()(GramInput, target)

In [0]:
#Dataset Processing
transform = tf.Compose([
    tf.Resize(512), #Default image_size
    #tf.Grayscale(num_output_channels=1),
    tf.ToTensor(), #Transform it to a torch tensor
    tf.Lambda(lambda x:x[torch.LongTensor([2, 1, 0])]), #Converting from RGB to BGR
    tf.Normalize(mean=[0.333, 0.333, 0.333], std=[0.5,0.5,0.5]), #subracting imagenet mean
    tf.Lambda(lambda x: x.mul_(255))
    ])

def load_img(path):
    img = Image.open(path)
    img = Variable(transform(img))
    img = img.unsqueeze(0)
    return img

def save_img(img):
    post = tf.Compose([
         tf.Lambda(lambda x: x.mul_(1./255)),
         tf.Normalize(mean=[-0.333, -0.333, -0.333], std=[1,1,1]),
         tf.Lambda(lambda x: x[torch.LongTensor([2,1,0])]), #turn to RGB
         ])
    img = post(img)
    img = img.clamp_(0,1)
    tutils.save_image(img,
                '%s/transfer_LBFGS_30_5cl_2.png' % ("./images"),
                normalize=True)
    return

def closure():
    optimizer.zero_grad()
    out = vgg(optimImg, loss_layers)
    totalLossList = []
    for i in range(len(out)):
        layer_output = out[i]
        loss_i = losses[i]
        target_i = targets[i]
        totalLoss = loss_i(layer_output, target_i) * weights[i]
        totalLossList.append(totalLoss)
    totalLoss = sum(totalLossList)
    totalLoss.backward()
    print('Loss: %f'%(totalLoss.data))
    return totalLoss


In [15]:
vgg_directory = "./drive/My Drive/UPenn_Courses/Fall2019/MachineLearning/Project/Images/vgg_conv.pth" #path to pretrained vgg vgg_directory
vgg = VGG(pooling = nn.AvgPool2d)
#print(vgg.state_dict())
vgg.load_state_dict(torch.load(vgg_directory))
for param in vgg.parameters():
    param.requires_grad = False

vgg.cuda() # Putting model on cuda

style_img = "./drive/My Drive/UPenn_Courses/Fall2019/MachineLearning/Project/Images/xingkong.jpg"
content_img = "./drive/My Drive/UPenn_Courses/Fall2019/MachineLearning/Project/Images/paris2.jpg"
styleImg = load_img(style_img)
contentImg = load_img(content_img)
styleImg = styleImg.cuda()
contentImg = contentImg.cuda()

style_weight = 1
content_weight = 9
content_layers = ['re12', 're22', 're32', 're42', 're52']
style_layers = ['re11','re21','re31','re41','re51']
style_Losses = [styleLoss()] * len(style_layers)
content_Losses = [nn.MSELoss()] * len(content_layers)

styleTargets = []
for t in vgg(styleImg, style_layers):
    t = t.detach()
    styleTargets.append(GramMatrix()(t))

contentTargets = []
for t in vgg(contentImg, content_layers):
    t = t.detach()
    contentTargets.append(t)

losses = style_Losses + content_Losses
targets = styleTargets + contentTargets
loss_layers = style_layers + content_layers
weights = [style_weight] * len(style_layers) + [content_weight] * len(content_layers)


#setting seed for pytorch
#torch.cuda.manual_seed_all(random.randint(1, 10000))
torch.cuda.manual_seed_all(1218)
if not os.path.exists("images/"):
    os.makedirs("images/")
#The below flag allows you to enable the cudnn auto-tuner
#to find the best algorithm for your hardware
cudnn.benchmark = True


optimImg = Variable(contentImg.data.clone(), requires_grad=True)
optimizer = optim.LBFGS([optimImg], lr=1, max_iter=30)
#optimizer = optim.SGD([optimImg], lr = 0.001, momentum=0.9)

#Shifting everything to cuda
for loss in losses:
    loss = loss.cuda()
optimImg.cuda()

# Training
epoches = 30

for epoch in range(epoches):
    print('Epoch: {}/{}'.format(epoch + 1, epoches))
    optimizer.step(closure)
print('Style transfer is completed')
outImg = optimImg.data[0].cpu()
save_img(outImg.squeeze())

Epoch: 1/30
Loss: 124752502784.000000
Loss: 124752502784.000000
Epoch: 2/30
Loss: 124752502784.000000
Loss: 19108452161537703936.000000
Loss: 124339101696.000000
Loss: 123966160896.000000
Loss: 123585486848.000000
Loss: 123198431232.000000
Loss: 122807205888.000000
Loss: 122413023232.000000
Loss: 122017513472.000000
Loss: 121621987328.000000
Loss: 69818916864.000000
Loss: 35102433280.000000
Loss: 20536731648.000000
Loss: 13926533120.000000
Loss: 10822856704.000000
Loss: 7230490112.000000
Loss: 5204071936.000000
Loss: 3806949632.000000
Loss: 3140289536.000000
Loss: 2351813120.000000
Loss: 2185562368.000000
Loss: 2003401344.000000
Loss: 1663170176.000000
Loss: 1392043136.000000
Loss: 1178405504.000000
Loss: 999624320.000000
Loss: 913091200.000000
Loss: 811279936.000000
Loss: 748297664.000000
Loss: 691478400.000000
Epoch: 3/30
Loss: 638517376.000000
Loss: 571293696.000000
Loss: 531270112.000000
Loss: 507755424.000000
Loss: 462096192.000000
Loss: 438166496.000000
Loss: 439088480.000000
Los